**I had kept my dataset on my google drive.
So I used google.colab's functionality for communicating with my google drive. 
I have commented that code below.**

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# from google.colab import files
# files.upload()

In [0]:
# # Run this cell to mount your Google Drive.
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
# !pip install -q kaggle && mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && ls ~/.kaggle && chmod 600 /root/.kaggle/kaggle.json

In [0]:
# kaggle competitions download -c recognizing-faces-in-the-wild -p /content/gdrive/My\ Drive/kaggle/BloodRelation

In [0]:
# !mkdir train  
# !mkdir test  
# !unzip -q train.zip -d train/
# !unzip -q test.zip -d test/ 

In [0]:
import numpy as np
import pandas as pd
import os
import torch
from fastai.vision import *
from torch import nn
from PIL import Image
import glob
import torchvision.transforms as transforms
from sklearn.metrics import *
import warnings
from functools import partial

import numpy as np
from scipy.sparse import csr_matrix
from scipy.stats import rankdata

from sklearn.utils import assert_all_finite
from sklearn.utils import check_consistent_length
from sklearn.utils import column_or_1d, check_array
from sklearn.utils.multiclass import type_of_target
from sklearn.utils.extmath import stable_cumsum
from sklearn.utils.sparsefuncs import count_nonzero
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.preprocessing import label_binarize

from sklearn.metrics.base import _average_binary_score


In [0]:
# os.chdir("gdrive/My Drive/kaggle/BloodRelation")

In [0]:
csv=pd.read_csv('train_relationships.csv')

In [0]:
trans=transforms.ToTensor()

In [0]:
def reshape_image(i1):
    i1.resize_((3,197, 197))
    return i1

In [0]:
dev = torch.device(
    "cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [0]:
def normalize_funcs(mean:FloatTensor, std:FloatTensor, do_x:bool=True, do_y:bool=False)->Tuple[Callable,Callable]:
    mean,std = tensor(mean),tensor(std)
    return (partial(_normalize_batch, mean=mean, std=std, do_x=do_x, do_y=do_y),
            partial(denormalize,      mean=mean, std=std, do_x=do_x))

def denormalize(x:TensorImage, mean:FloatTensor,std:FloatTensor, do_x:bool=True)->TensorImage:
    return x.cpu().float()*std[...,None,None] + mean[...,None,None] if do_x else x.cpu()

def _normalize_batch(b:Tuple[Tensor,Tensor], mean:FloatTensor, std:FloatTensor, do_x:bool=True, do_y:bool=False)->Tuple[Tensor,Tensor]:
    x,y = b
    mean,std = mean.to(x.device),std.to(x.device)
    if do_x: x = normalize(x,mean,std)
    if do_y and len(y.shape) == 4: y = normalize(y,mean,std)
    return x,y

In [0]:
class MyDataset(Dataset):
    def __init__(self, Df, istest=False):
        self.istest=istest
        if not self.istest:
            self.X1=list(Df.p1)
            self.X2=list(Df.p2)
            self.y=list(Df.Relativity)
            
        else:
            self.Df=Df
                

    def __len__(self):
        if not self.istest:
            return len(self.X1)
        else:
            return len(self.Df.img_pair)
    
    def batch_stats(self, funcs:Collection[Callable]=None, ds_type:DatasetType=DatasetType.Train)->Tensor:
        funcs = ifnone(funcs, [torch.mean,torch.std])
        x = self.one_batch(ds_type=ds_type, denorm=False)[0].cpu()
        return [func(channel_view(x), 1) for func in funcs]
    
    def normalize(self, stats:Collection[Tensor]=None, do_x:bool=True, do_y:bool=False)->None:
        if getattr(self,'norm',False): raise Exception('Can not call normalize twice')
        if stats is None: self.stats = self.batch_stats()
        else:             self.stats = stats
        self.norm,self.denorm = normalize_funcs(*self.stats, do_x=do_x, do_y=do_y)
        return self
             
    def __getitem__(self, idx):
        if self.istest:
            a,b=self.Df.img_pair[idx].split("-")
            return [reshape_image(trans(Image.open("test/"+a))), reshape_image(trans(Image.open("test/"+b)))],1
        return [reshape_image(trans(Image.open(self.X1[idx]))), reshape_image(trans(Image.open(self.X2[idx])))] , torch.tensor(float(self.y[idx]),dtype=torch.float)

In [0]:
class MultInputNN(nn.Module):
    def __init__(self):
        super(MultInputNN,self).__init__()
        #self.model=models.resnet50(pretrained=True,)
        self.model=models.vgg19_bn(pretrained=True)
        
        self.model2=nn.Sequential(
            nn.Linear(666,256),
            nn.Tanh(),
            nn.Linear(256,100),
            nn.ReLU(),
            nn.Dropout(0.01),
            nn.Linear(100,1),
            nn.Sigmoid()
        )
        
        self.MaxPool2d=nn.Sequential(nn.MaxPool2d(3))
        self.AvgPool2d=nn.Sequential(nn.AvgPool2d(3))
        for params in self.MaxPool2d:
            params.requires_grads=True
        for params in self.AvgPool2d:
            params.requires_grads=True     
        for params in self.model.parameters():
            params.requires_grads=True
        for params in self.model2:
            params.requires_grads=True
        
         
    def forward(self, input1, input2):
        c1=self.model(input1)
        c2=self.model(input2)
        c1=c1[...,None]
        c2=c2[...,None]        
       
        a2=self.AvgPool2d(c1)
        a4=self.AvgPool2d(c2)
        c1_=torch.pow(a2,2)
        c2_=torch.pow(a4,2)
        c3=torch.sub(c1_,c2_)
        c_=torch.sub(a2,a4)
        c4=torch.pow(c_,2)
        combined = torch.cat((c4.view(c1.size(0), -1),
                          c3.view(c2.size(0), -1)), dim=1)
        output=self.model2(combined)
        return output

In [0]:
Df=pd.read_csv("Relations.csv")


In [0]:
# The accuracy metrics

def Accuracy(input, targs):
    n = targs.shape[0]
    
    targs = targs.view(n,-1)
    for i in range(len(input)):
        if input[i]>0.5:
            input[i]=1
        else:
            input[i]=0
    for i in range(len(targs)):
        if targs[i]>0.5:
            targs[i]=1
        else:
            targs[i]=0
        
    return torch.tensor((1-(input==targs).float().mean()),requires_grad=True)


In [0]:
# The Loss function which kaggle was going to use for ranking

def Roc_auc_score2(score, true, average="macro", sample_weight=None,
                  max_fpr=None):
    
    y_score=torch.tensor(score)
    y_true=torch.tensor(true)    

    def _binary_roc_auc_score(y_true, y_score, sample_weight=None):

        fpr, tpr, _ = roc_curve(y_true, y_score,
                                sample_weight=sample_weight)
        if max_fpr is None or max_fpr == 1:
            return auc(fpr, tpr)
        if max_fpr <= 0 or max_fpr > 1:
            raise ValueError("Expected max_fpr in range (0, 1], got: %r"
                             % max_fpr)


        stop = np.searchsorted(fpr, max_fpr, 'right')
        x_interp = [fpr[stop - 1], fpr[stop]]
        y_interp = [tpr[stop - 1], tpr[stop]]
        tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
        fpr = np.append(fpr[:stop], max_fpr)
        partial_auc = auc(fpr, tpr)

        # McClish correction: standardize result to be 0.5 if non-discriminant
        # and 1 if maximal
        min_area = 0.5 * max_fpr**2
        max_area = max_fpr
        return torch.tensor((1 - (0.5 * (1 + (partial_auc - min_area) / (max_area - min_area)))),requires_grad=True)

    y_type = type_of_target(y_true)
    if y_type == "binary":
        labels = np.unique(y_true)
        y_true = label_binarize(y_true, labels)[:, 0]

    return torch.tensor((1 - _average_binary_score(
        _binary_roc_auc_score, y_true, y_score, average,
        sample_weight=sample_weight)),requires_grad=True)


In [0]:
train_data=MyDataset(Df[:249000])
valid_data=MyDataset(Df[249000:])

In [0]:
# Run this to have get data

test_df=pd.read_csv("sample_submission.csv")
test_ds=MyDataset(test_df,istest=True)
data_bunch=DataBunch.create(train_data,valid_data ,test_ds=test_ds,bs=16,device=dev)

In [0]:
data_bunch.normalize(imagenet_stats)

In [0]:
learn=Learner(data_bunch,MultInputNN(),loss_func=Roc_auc_score2,metrics=[Roc_auc_score2,Accuracy])

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(1,2.5e-3,wd=0.03,callbacks=[callbacks.ReduceLROnPlateauCallback(learn,monitor='trn_loss',mode='max'),callbacks.SaveModelCallback(learn,mode='max',name='ModelVggPreTrained-2')])

In [0]:
def save_preds(res,name):
    r=res[0]
    l=[]
    for i in r:
        l.append(i[0])
    sub=pd.read_csv("sample_submission.csv")
    sub.is_related=l
    for i in range(len(sub.is_related)):
        sub.is_related[i]=float(sub.is_related[i])
    sub.to_csv(name,index=False)
    return sub
    

In [0]:
save_preds(res,'VggPreds2.csv')